In [17]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras.models import Sequential,load_model
from keras.layers import Embedding,Dense,Flatten
#from keras.models import load_model
import numpy as np
import os,sys
import math
import random
seed_=10

In [18]:
label_dict_static={}
def create_static_label():
    
    with open("/home/ubuntu/composite.txt") as file:
        for _ in range(18227):
            line  = file.readline().strip().split(",")
            label_dict_static[line[0]] = line[1]
create_static_label()



In [19]:
word2vec_model_static = KeyedVectors.load("/home/ubuntu/first.model", mmap='r')
#word2vec_model_dynamic = KeyedVectors.load("/home/ubuntu/dynamic_word.model", mmap='r')

In [20]:
import os
#nb_words=len(word2vec_model.wv.vocab), will be parameter for tokenizer
tokenizer = Tokenizer(filters='#\n')
tokenizer.fit_on_texts(word2vec_model_static.wv.vocab.keys())
word_index = tokenizer.word_index
temp_array=[]
validation_size = 50
def get_validation_data(file_names):
    c=0 
    temp_array=[]
    
    while True:
        if c>=len(file_names):
            c=0
        for i,j in enumerate(file_names[c:c+validation_size]):
            l = open("/home/ubuntu/static_validation/"+j,'r').read().strip()
            padded_sequence =  sequence.pad_sequences(tokenizer.texts_to_sequences([l]),maxlen=4000,padding='post',truncating='post')
            temp_array.append(padded_sequence[0])
            

        temp_array = np.array(temp_array)
        #label_array = np.array(label_array)
        #temp_array = temp_array[..., np.newaxis]
        #print("\nYIELDING FROM c = ",c," c+validation_size = ",c+validation_size," and length of temp_array = ",len(temp_array))
        yield (temp_array)
        temp_array=[]
        c+=validation_size
list_of_validation_files = os.listdir("/home/ubuntu/static_validation")
#print(list_of_validation_files)
list_of_validation_files.sort()
random.seed(a=seed_)
random.shuffle(list_of_validation_files)
print(list_of_validation_files[333])
print(list_of_validation_files[2112])
print(list_of_validation_files[454])
validation_generator = get_validation_data(list_of_validation_files)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


AACF906F90188F32700545FCA195CC045D3FF3A9257BB64CC841C7B8BB37B792.txt
740585503e64a66f33e94f88b675b2f2.txt
AD1939742F08BF7793842C629A1BE52884F242BB730C8A765A068A2B89CA12DA.txt


In [21]:
static_model=load_model("/home/ubuntu/static-keras/lstm_model-final-mask_zero.h5")
#dynamic_model = load_model("/home/ubuntu/dynamic-keras/final-dynamic_lstm_model-maskZero.h5")

In [22]:
prediction_static_original = static_model.predict_generator(validation_generator,math.ceil(len(list_of_validation_files)/validation_size))


In [23]:
prediction_static=[]
malicious_count=0
desired_malicious_count=400
index_array=[]
mal_c=0
ben_c=0
for index, prediction in enumerate(prediction_static_original):
    if int(label_dict_static.get(list_of_validation_files[index]))==1 and malicious_count<desired_malicious_count:
        prediction_static.append(prediction)
        index_array.append(index)
        mal_c+=1
        malicious_count+=1
    if int(label_dict_static.get(list_of_validation_files[index]))==0:
        prediction_static.append(prediction)
        index_array.append(index)
        ben_c+=1
    
print(len(prediction_static))
print(len(index_array))
print(mal_c,ben_c)

2400
2400
400 2000


In [24]:
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.clf()
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",verticalalignment="top",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('Confusion-Matrix-Static-'+str(desired_malicious_count)+'png')
    plt.clf()


In [25]:
from sklearn.metrics import confusion_matrix
actual_labels=[]
for i,j in enumerate(list_of_validation_files):
    if i in index_array:
        if int(label_dict_static.get(j))==1:
                    actual_labels.append(1)
        else:
                    actual_labels.append(0)
#print(actual_labels)
#print(prediction)
prediction=prediction_static
#prediction = (np.asarray(prediction_dynamic)+np.asarray(prediction_static))/2
print(prediction[:3])
# print(prediction_dynamic[:3])
# print(prediction_static[:3])
prediction1 = np.argmax(prediction, axis=1) 
print(prediction1)

cm = confusion_matrix(actual_labels, prediction1)
plot_confusion_matrix(cm,['benign','malicious'],title='CONFUSION MATRIX')

[array([0.15612999, 0.84387   ], dtype=float32), array([0.9952608 , 0.00473927], dtype=float32), array([9.335336e-06, 9.999907e-01], dtype=float32)]
[1 0 1 ... 0 0 0]
Confusion matrix, without normalization
[[1949   51]
 [  38  362]]


<Figure size 432x288 with 0 Axes>

In [26]:

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from funcsigs import signature
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score
from inspect import signature

def call_precision_recall_curve(truelabel,predictedlabel,actualprediction,label):
    
    #print(truelabel.shape)
    #print(actualprediction.shape)
    
    precision, recall, _ = precision_recall_curve(truelabel,actualprediction)
    au = auc(recall, precision)
    print("Area under graph "+str(au))
    print("-------------------------------------------------")
    rec_score = recall_score(truelabel,predictedlabel,pos_label=1,average='binary')
    precise_score = precision_score(truelabel,predictedlabel,pos_label=1,average='binary')
    f1 = f1_score(truelabel,predictedlabel,pos_label=1, average='binary')
    print("Precision for Malicious apps "+str(precise_score))
    print("Recall for Malicious apps "+str(rec_score))
    print("F1-score for Malicious apps  " + str(f1))
    print("-------------------------------------------------")
    rec_score = recall_score(truelabel,predictedlabel,pos_label=0,average='binary')
    precise_score = precision_score(truelabel,predictedlabel,pos_label=0,average='binary')
    f1 = f1_score(truelabel,predictedlabel,pos_label=0, average='binary')
    print("Precision for Benign apps "+str(precise_score))
    print("Recall for Benign apps "+str(rec_score))
    print("F1-score for Benign apps  " + str(f1))
    
    
    #plot the no-skill line too
    positive_cases = sum(truelabel)/len(truelabel)
    plt.plot([0, 1], [positive_cases, positive_cases], linestyle='--')
    
    
    
    # In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve')
    plt.tight_layout()
    plt.savefig('Precision-Recall-Curve-Static'+str(desired_malicious_count)+'.png')
    plt.clf()
prediction_for_1=[]
for each_prediction in prediction:
    prediction_for_1.append(each_prediction[1])
call_precision_recall_curve(np.array(actual_labels),np.array(prediction1),np.array(prediction_for_1),1)

Area under graph 0.9511380003807398
-------------------------------------------------
Precision for Malicious apps 0.8765133171912833
Recall for Malicious apps 0.905
F1-score for Malicious apps  0.8905289052890528
-------------------------------------------------
Precision for Benign apps 0.980875691997987
Recall for Benign apps 0.9745
F1-score for Benign apps  0.9776774517180838


<Figure size 432x288 with 0 Axes>